# Scikit-learn

This notebook is an introduction to [scikit-learn](https://scikit-learn.org/stable/), the main package for machine learning (excluding neural nets).

Most algorithms are available as objects with common methods like ``fit``, ``predict`` or ``transform``. 

The outline is the following:

**1. Supervised learning**

* classification
* cross-validation
* grid search
* feature selection
* soft classification
* regression

**2. Unsupervised learning**

* clustering
* anomaly detection
* dimensionality reduction

**3. Feature engineering**

* scaling
* categorical features
* text features

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder, OrdinalEncoder, LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits, load_wine
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer

## 1. Supervised learning

The objective of supervised learning is to map data ``X`` to target ``y``.

``algo.fit(X, y)``

## Classification

In [ ]:
iris = sns.load_dataset('iris')

In [ ]:
iris.info()

In [ ]:
sns.pairplot(data=iris, hue="species", height=2);

In [ ]:
# data
X = iris.drop(columns=['species'])
y = iris['species']

feature_names = np.array(X.columns)
X = np.array(X)
y = np.array(y)

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
algo = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# fit to training data
algo.fit(X_train, y_train)

In [ ]:
# predict on test data
y_pred = algo.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
labels = np.unique(y_test)

In [ ]:
confusion = confusion_matrix(y_test, y_pred, labels=labels)

In [ ]:
sns.heatmap(confusion, square=True, annot=True, cmap='binary', xticklabels=labels, yticklabels=labels)
plt.ylabel('Truth')
plt.xlabel('Prediction');

## Cross-validation

In [ ]:
# data
X = iris.drop(columns=['species']).to_numpy()
y = iris['species'].to_numpy()

In [ ]:
algo = KNeighborsClassifier(n_neighbors=3)

In [ ]:
# 5 splits of data
cross_val_score(algo, X, y, cv=5)

## Grid search

Search for the best parameters of the model.

### Single algorithm

In [ ]:
algo = KNeighborsClassifier()
parameters = {'n_neighbors': np.arange(1, 10)}
grid = GridSearchCV(algo, parameters, cv=5)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
y_pred = grid.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
grid.best_params_

### Multiple algorithms

In [ ]:
algos = [KNeighborsClassifier(), 
         SVC(), 
         RandomForestClassifier()]

In [ ]:
# corresponding parameters
parameters = [{'n_neighbors': np.arange(1, 10)}, 
              {'C': [0.5, 1, 1.5], 'kernel': ['linear', 'poly']}, 
              {}]

In [ ]:
scores = []
for algo, param in zip(algos, parameters):
    grid = GridSearchCV(algo, param, cv=5)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

In [ ]:
scores

## Feature selection

In [ ]:
# select from a model
model = RandomForestClassifier()
algo = SelectFromModel(model)

In [ ]:
algo.fit(X_train, y_train)

In [ ]:
features = algo.get_support()

In [ ]:
print(feature_names[features])

In [ ]:
X_train_ = algo.transform(X_train)
X_test_ = algo.transform(X_test)

In [ ]:
algo = KNeighborsClassifier()
parameters = {'n_neighbors': np.arange(1, 10)}
grid = GridSearchCV(algo, parameters, cv=5)

In [ ]:
grid.fit(X_train_, y_train)

In [ ]:
y_pred = grid.predict(X_test_)

In [ ]:
accuracy_score(y_test, y_pred)

## Soft classification

In [ ]:
algo = SVC(probability=True)

In [ ]:
algo.fit(X_train, y_train)

In [ ]:
probs = algo.predict_proba(X_test)

In [ ]:
probs.shape

In [ ]:
confidence = np.max(probs, axis=1)

In [ ]:
sns.histplot(confidence, bins=10);

In [ ]:
dims = [2, 3]
for label in labels:
    mask = y_test==label
    plt.scatter(X_test[mask,dims[0]], X_test[mask,dims[1]], s=200*(confidence[mask] - 0.5))
plt.xlabel(iris.columns[dims[0]])
plt.ylabel(iris.columns[dims[1]])
plt.show()

## Regression

### Linear regression

In [ ]:
# data
x = np.linspace(0, 10, 20)
y = 2 * x - 10
y += 2 * np.random.randn(len(x))

In [ ]:
plt.scatter(x, y);

In [ ]:
algo = LinearRegression()

In [ ]:
# shape = (n_samples, n_features)
X = x.reshape(-1, 1)

In [ ]:
algo.fit(X, y)

In [ ]:
y_pred = algo.predict(X)

In [ ]:
plt.scatter(x, y)
plt.plot(x, y_pred, color='k');

### Polynomial regression

In [ ]:
# data
x = np.linspace(0, 10, 50)
x[x==0] = 10**-6
y = np.sin(x) / x
y += 0.1 * np.random.randn(len(x))

In [ ]:
plt.scatter(x, y);

In [ ]:
algo = make_pipeline(PolynomialFeatures(5), LinearRegression())

In [ ]:
X = x.reshape(-1,1)

In [ ]:
algo.fit (X, y)

In [ ]:
y_pred = algo.predict(X)

In [ ]:
plt.scatter(x, y)
plt.plot(x, y_pred, color='k');

## 2. Unsupervised learning

The objective of unsupervised learning is to understand the data structure ``X`` **without** target ``y``.

``algo.fit(X)``

## Clustering

In [ ]:
X = iris.drop(columns=['species']).to_numpy()
y = iris['species'].to_numpy()

In [ ]:
n_clusters = 3
algo = KMeans(n_clusters)

In [ ]:
y_pred = algo.fit_predict(X[:, features])

In [ ]:
dims = np.where(features)[0]
colors = ['b', 'r', 'g']
for i in range(n_clusters):
    mask = y_pred==i
    plt.scatter(X[mask,dims[0]], X[mask,dims[1]])
plt.xlabel(iris.columns[dims[0]])
plt.ylabel(iris.columns[dims[1]])
plt.show()

## Anomaly detection

In [ ]:
algo = IsolationForest(contamination=0.05)

In [ ]:
normal = algo.fit_predict(X[:, features])

In [ ]:
dims = np.where(features)[0]
colors = ['b', 'r', 'g']
for label, color in zip(labels, colors):
    mask = (y==label) & (normal > 0)
    plt.scatter(X[mask,dims[0]], X[mask,dims[1]], color=color)
    mask = (y==label) & (normal < 0)
    plt.scatter(X[mask,dims[0]], X[mask,dims[1]], color=color, alpha=0.5)
plt.xlabel(iris.columns[dims[0]])
plt.ylabel(iris.columns[dims[1]])
plt.show()

## Dimensionality reduction

In [ ]:
digits = load_digits()

In [ ]:
X = digits.data
y = digits.target

In [ ]:
X.shape

In [ ]:
def show_digits(X, y):
    img = np.zeros((100, 100))
    for i in range(10):
        index = np.where(y == i)[0][:10]
        for j, sample in enumerate(index):
            img[i*10+1:i*10+9,j*10+1:j*10+9] = X[sample].reshape((8, 8))
    plt.imshow(img, cmap='binary')
    plt.xticks([])
    plt.yticks(5 + 10*np.arange(10), np.arange(10))

In [ ]:
show_digits(X, y)

In [ ]:
algo = PCA(n_components=8)

In [ ]:
X_reduced = algo.fit_transform(X)

In [ ]:
X_reduced.shape

In [ ]:
plt.bar(np.arange(8) + 1, algo.explained_variance_ratio_);

In [ ]:
X_inv = algo.inverse_transform(X_reduced)

In [ ]:
show_digits(X_inv, y)

## 3. Feature engineering

The objective of feature engineering is to transform data into a proper array of numerical values ``X``.

## Scaling

In [ ]:
algo = StandardScaler()

In [ ]:
wine = load_wine()

In [ ]:
X = wine.data
y = wine.target

In [ ]:
sns.catplot(data=pd.DataFrame(X));

In [ ]:
X_scale = algo.fit_transform(X)

In [ ]:
sns.catplot(data=pd.DataFrame(X_scale));

In [ ]:
# application
algo = KNeighborsClassifier(n_neighbors=3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [ ]:
algo.fit(X_train, y_train)

In [ ]:
algo.score(X_test, y_test)

In [ ]:
# scaling
pipe = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=3))

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.score(X_test, y_test)

## Categorical features

In [ ]:
df = sns.load_dataset("titanic")

In [ ]:
df = df.drop(columns=["deck", "embark_town", "class", "alive"])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df["embarked"] = df["embarked"].fillna("Unknown")
df["age"] = df["age"].fillna(-1)

In [ ]:
y = np.array(df["survived"])
df = df.drop(columns="survived")

In [ ]:
algo = OrdinalEncoder()

In [ ]:
X = algo.fit_transform(df)

In [ ]:
X.shape

In [ ]:
for i, col in enumerate(df.columns):
    print(col, len(set(X[:, i])))

In [ ]:
algo = OneHotEncoder()

In [ ]:
X_bin = algo.fit_transform(df)

In [ ]:
X_bin.shape

In [ ]:
X_bin

In [ ]:
categories = algo.categories_

In [ ]:
{col: len(cat) for col, cat in zip(df.columns, categories)}

In [ ]:
# specific columns
algo = make_column_transformer((OneHotEncoder(sparse=False), ['embarked', 'who']))

In [ ]:
X_bin = algo.fit_transform(df)

In [ ]:
X_bin.shape

In [ ]:
X_bin[0]

In [ ]:
df.head()

In [ ]:
# keep others
tf = make_column_transformer((OneHotEncoder(drop='if_binary', sparse=False), 
                                ['sex','embarked', 'who', 'adult_male', 'alone']), remainder='passthrough')

In [ ]:
X_bin = tf.fit_transform(df)

In [ ]:
X_bin.shape

In [ ]:
algo = RandomForestClassifier()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [ ]:
algo.fit(X_train, y_train)

In [ ]:
algo.score(X_test, y_test)

## Text features

In [ ]:
algo = CountVectorizer()

In [ ]:
X = algo.fit_transform([digits.DESCR, wine.DESCR])

In [ ]:
X.shape

In [ ]:
counts = X.toarray().sum(axis=0)

In [ ]:
features_names = np.array(algo.get_feature_names())

In [ ]:
# top words
features_names[np.argsort(-counts)[:5]]